In [ ]:
import torch
import torchvision
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data.dataset import Dataset
!pip install torchsummary
from torchsummary import summary
import matplotlib.pyplot as plt
from torch import nn, optim
import torch.nn.functional as F
import time
import cv2 as cv

In [ ]:
class DRDataset(Dataset):
    def __init__(self, csv_path):
        # Transforms
        self.to_tensor = transforms.ToTensor()
        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        single_image_name = '/scratch/groups/rubin/rvignav/input/train/' + self.image_arr[index] + '.jpeg'
        img_as_img = Image.open(single_image_name)
        img_as_img = img_as_img.resize((400,300))
        
        gamma = 1.3
        lookUpTable = np.empty((1,256), np.uint8)
        for i in range(256):
            lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
        img_as_img = cv.LUT(np.uint8(img_as_img), lookUpTable)

        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len

In [ ]:
full_dataset = DRDataset('/scratch/groups/rubin/rvignav/input/train/trainLabels.csv')

train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset)) 
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64,
                                          shuffle=True)

# print(len(train_loader.dataset))
# print(len(val_loader.dataset))
# print(len(test_loader.dataset))

In [ ]:
"""
0 - No DR
1 - Mild
2 - Moderate
3 - Severe
4 - Proliferative DR
"""
classes = ('none', 'mild', 'moderate', 'severe', 'proliferative')

In [ ]:
for data, target in train_loader:
    for i in range(0, 4):  
        im = data[i]
        im = torch.squeeze(im)
        plt.imshow(np.transpose(im.numpy(), (1, 2, 0)), cmap='gray')
        plt.show()
        print(target[i] + ": " + classes[(int)(target[i])])
    break

In [ ]:
# https://amaarora.github.io/2020/06/29/FocalLoss.html
class WeightedFocalLoss(nn.Module):
    "Non weighted version of Focal Loss"
    def __init__(self, alpha=.25, gamma=2):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = torch.tensor([alpha, 1-alpha])
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        targets = targets.type(torch.long)
        at = self.alpha.gather(0, targets.data.view(-1))
        pt = torch.exp(-BCE_loss)
        F_loss = at*(1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

In [ ]:
network = torchvision.models.vgg16() 
criterion = WeightedFocalLoss() #nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=0.005, momentum=0.5, weight_decay=1e-4)

In [ ]:
from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 512)),
                           ('relu', nn.ReLU()), 
                           ('dropout', nn.Dropout(p=0.337)),
                           ('fc2', nn.Linear(512, 5)),
                           ('output', nn.LogSoftmax(dim=1))
                             ]))

network.classifier = classifier

summary(network, (3, 300, 300))

In [ ]:
def train(model, loader, criterion):
    model.train()
    current_loss = 0
    current_correct = 0
    for train, t in iter(loader):
        y_train = np.zeros(len(t))
        for i in range(len(t)):
            y_train[i] = int(t[i])
        y_train = torch.from_numpy(y_train).long()
        optimizer.zero_grad()
        output = model.forward(train)
        _, preds = torch.max(output,1)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        current_loss += loss.item()*train.size(0)
        current_correct += torch.sum(preds == y_train.data)
    epoch_loss = current_loss / len(train_loader.dataset)
    epoch_acc = current_correct.double() / len(train_loader.dataset)
        
    return epoch_loss, epoch_acc

In [ ]:
def validation(model, loader, criterion):
    model.eval()
    valid_loss = 0
    valid_correct = 0
    for valid, t in iter(loader):
        y_valid = np.zeros(len(t))
        for i in range(len(t)):
            y_valid[i] = int(t[i])
        y_valid = torch.from_numpy(y_valid).long()
        output = model.forward(valid)
        valid_loss += criterion(output, y_valid).item()*valid.size(0)
        equal = (output.max(dim=1)[1] == y_valid.data)
        valid_correct += torch.sum(equal)#type(torch.FloatTensor)
    
    epoch_loss = valid_loss / len(val_loader.dataset)
    epoch_acc = valid_correct.double() / len(val_loader.dataset)
    
    return epoch_loss, epoch_acc

In [ ]:
for param in network.parameters():
    param.require_grad = False
#train and validate
epochs = 20  
epoch = 0
#send model to GPU

for e in range(epochs):
    epoch +=1
    print(epoch)
    with torch.set_grad_enabled(True):
        epoch_train_loss, epoch_train_acc = train(network,train_loader, criterion)
    print("Epoch: {} Train Loss : {:.4f}  Train Accuracy: {:.4f}".format(epoch,epoch_train_loss,epoch_train_acc))
    if e % 5 == 0:
        torch.save(network.state_dict(), '/scratch/groups/rubin/rvignav/checkpoint_' + str(e // 5) + '.pth')
with torch.no_grad():
    epoch_val_loss, epoch_val_acc = validation(network, val_loader, criterion)
    print("Epoch: {} Validation Loss : {:.4f}  Validation Accuracy {:.4f}".format(epoch,epoch_val_loss,epoch_val_acc))

In [ ]:
network.eval()
total = 0
correct = 0 
count = 0

for test, t in iter(test_loader):
    y_test = np.zeros(len(t))
    for i in range(len(t)):
        y_test[i] = int(t[i])
    y_test = torch.from_numpy(y_test)
    with torch.no_grad():
        output = network.forward(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item() 
        count += 1

print("Accuracy of network on test images is ... {:.4f}....count: {}".format(100*correct/total,  count ))

In [ ]:
torch.save(network.state_dict(), '/scratch/groups/rubin/rvignav/dr_not_stylized.pth')

In [ ]:
for data, target in val_loader:
    outputs = network(data)
    _, predicted = torch.max(outputs, 1)
    for i in range(0, 4):  
        im = data[i]
        im = torch.squeeze(im)
        plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))
        plt.show()
        print("TARGET-- " + target[i] + ": " + classes[(int)(target[i])])
        # print(predicted[i].numpy())
        print("PREDICTION-- " + str(predicted[i].numpy()) + ": " + classes[predicted[i].numpy()])
    break